# Resumo do código


>### Entra: 1_Ninjas e 2_Delta  
>### Sai: OUT_2_DataFusion_2022
---
O objectivo é receber dados dos Ninjas e da Delta e devolver um ficheiro Wide que permita verificar o comportamento de vários produtos em diversas lojas através de representações gráficas. 

Devolve um ficheiro que entra no R para os dados de 2022

---
> Input
> - Única diferença é coluna Id que deve corresponder à ordem dos dias (porque os dias homólogos não coincidem no número)

> Output
> - O mesmo de 2PilotoWide, embora nem todas as métricas sejam necessárias

In [41]:
import pandas as pd
import numpy as np

def escrever_excel(dfa, nome):
    dfa.to_excel('%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv(nome+'.csv', index=False)

# Ler

- # `Ninjas (sinal)`

In [42]:
#Info Ninja
dfNinjas=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\1Ninjas_Limpo.xlsx")

> Definir variáveis importantes
> - Se houver um ficheiro com os produtos

In [43]:
# Ler ficheiro para dataframe
df_produtos = pd.read_csv('D:\\B&N Dados\\Delta\\Padrão_05_2023\\produtos.txt', header=None)

# Passar para uma lista
produtos = df_produtos[0].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Se se quiser usar o primeiro e o último produtos

# Nomes de entrada
x= input("Primeiro produto: ")                # CAFÉ DELTA Q MYTHIQ 80CAP
y= input("Último produto: ")                  # BEBIDA CEREAIS DELTA C/20%CAFE FR 200G

# Passar para uma lista
produtos = dfNinjas.columns[dfNinjas.columns.get_loc(x): dfNinjas.columns.get_loc(y)+1].tolist()
resto = dfNinjas.columns.difference(produtos)

> - Semana

dfNinjas['Semana'] = dfNinjas['DATA'].dt.isocalendar().week

dfNinjas['Semana'] = dfNinjas.groupby('Semana').ngroup() + 1

> - Id

In [44]:
# Extract the day and month components from the 'datetime' column
dfNinjas['day'] = pd.to_datetime(dfNinjas['DATA']).dt.day
dfNinjas['month'] = pd.to_datetime(dfNinjas['DATA']).dt.month

# Create a new column combining day and month
dfNinjas['day_month'] = dfNinjas['day'].astype(str) + '-' + dfNinjas['month'].astype(str)

# Assign unique IDs to each unique day-month combination
dfNinjas['id'], _ = pd.factorize(dfNinjas['day_month'])

dfNinjas = dfNinjas.drop(columns=['day', 'month', 'day_month'])

- #  `Delta (stocks, reposição e sellouts)`

> Stocks

In [46]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão_05_2023\\2_Delta_2022Id.xlsx", sheet_name="Sheet1")

# VER

In [47]:
#dfNinjas.head()
#dfDelta.info()

# <font color=green>Mergir</font>

In [37]:
# Dataframe mergido no fim
dfMeio=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["id", "STORE", "SOH", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    dfMeio=pd.merge(dfMeio, dfStocks, how="left", on = ["id","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio=dfMeio.rename(columns={"SOH": "STOCK_1_Dias_Antes %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
    

In [39]:
dfFinal=dfMeio.copy()

### <font color=green>Fim</font>

# Já temos DataFrame com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição

## Fazer:

- Rotura
- Sinal -> Avaliação da informação ninja vs stock
- Ciclos
- Adequação de Stock

> Rotura

In [165]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal[rotura] = np.where((dfFinal[stock] > 0) & ~(dfFinal[i].isna()), 0, 
                      np.where((dfFinal[stock] <= 0) & (dfFinal[preslinear] > 0) & ~(dfFinal[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [166]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal[pre_rotura] = (dfFinal[stock] < dfFinal[preslinear]).astype(int)

>Sinal

In [167]:
#dfFinal["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Verdadeiramente Correcto",
           2: "Falso Positivo",
           3: "Falso Negativo",
           4: "Rejeição Correcta"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    dfFinal[sinal] = np.where((dfFinal[stock] > 0) & (dfFinal[i] == 1), "Verdadeiramente Correcto",
                     np.where((dfFinal[stock] > 0) & (dfFinal[i] == 0), "Falso Positivo",
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 1), "Falso Negativo",
                     np.where((dfFinal[stock] <= 0) & (dfFinal[i] == 0), "Rejeição Correcta", np.nan))))

    # Map the numbers to the corresponding strings
    dfFinal[sinal] = dfFinal[sinal].map(mapping)
    
    

# 1: Há Stock e Sinal Positivo Ninja
# 2: Há Stock e Sinal Negativo Ninja
# 3: Não há Stock e Sinal Positivo Ninja
# 4: Não há Stock e Sinal Negativo Ninja

> Ciclos

In [168]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal[ciclos]=dfFinal[stock]/dfFinal[linear]

> Adequação

In [169]:
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal[adequa] = np.where(dfFinal[ciclos] > 1.1, "Stock Suficiente",
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] >= dfFinal[linear]), "Stock Insuf c Forn Adequado",
                      np.where((dfFinal[ciclos] <= 1.1) & (dfFinal[transito] + dfFinal[esperado] + dfFinal[stock] < dfFinal[linear]), "Stock Insuf c Forn Desadequado", 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal[adequa] = dfFinal[adequa].map(mapping)

- Dias para a rotura

In [170]:
for i in produtos:
    stock  = "STOCK %s" % i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal[dias_rotura]=dfFinal[stock]/dfFinal[sellout]

- Dias para a rotura de prateleira

In [171]:
for i in produtos:
    linear = "PRESLINEAR %s" % i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal[dias_rotura] = dfFinal[linear] / dfFinal[sellout]

- Balanço

In [172]:
for i in produtos:
    balanço = "BALANCE %s" % i
    stock  = "STOCK %s" % i
    sellout = "SELLOUT %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    
    dfFinal[balanço] = dfFinal[sellout] / (dfFinal[stock] + dfFinal[transito] + dfFinal[esperado])

# Acabamos com:
- Informação Ninjas
- Informação Stocks, Linear, Intransit e Expected
- Informação Sellouts, Vendedor, Reposição
- Rotura
- Pré-Rotura
- Sinal 
- Ciclos
- Adequação de Stock
- Dias para a rotura normal
- Dias para a rotura móvel XXXXXXXXXXX
- Dias para a rotura de prateleira
- Balanço

# Escrever

In [40]:
escrever_excel(dfFinal, "OUT_2_DataFusion_2022")
#dfFinalCorr.to_csv('ficheiro_NSS.csv', index=False)